In [1]:
                                             #REQUESTS WITH BEAUTIFULSOUP
#IMPORTING PACKAGES
import requests
from bs4 import BeautifulSoup
import pandas as pd
#URL
base_URL = "https://www.ekb-storage.de/"
URL = "https://www.ekb-storage.de/en/ekb-storage/storage-usage/"
page = requests.get(URL)
#PARSE HTML CONTENT
soup = BeautifulSoup(page.content, "html.parser")
#SCRAPPING WEB PAGES
def scrap():
    results = soup.find_all(class_ = "btn btn-primary w-100") #CLICK OF A BTN
    soup.find_all(class_="table-responsive mt-4") #ENTIRE TABLE
    data=soup.find_all('tbody') #TABLE ROW CONTENT
    l=[]
    for i in data:
        res=i.get_text().strip().replace("\n",' ')
        res=res.split('  ')
        for j in res:
            result=j.lstrip().split(' ')
            result.append(URL)
            l.append(result)
    #DATAFRAME
    df = pd.DataFrame(l, columns = ["Date","Injection(kWh)","Withdrawal(kWh)","Storage(kWh)","URL"])
    df['SNO'] = df.index + 1
    df.set_index('SNO',inplace=True)
#print(l)
    return df

In [5]:
                                                 #FLASK API WITH SQLITE
#IMPORTING PACKAGES
import os
import sqlite3
from flask import jsonify
from flask_api import FlaskAPI
from flask_restful import Api
from sqlalchemy import create_engine
from flask import request
import json
#INSTANTIATE AND INITIALIZE FLASK APPLICATION
app=FlaskAPI(__name__)
api=Api(app)
#CREATE A ENGINE FOR DATABASE
c=create_engine('sqlite:///SQLITESAPI.db')
#----------------------DEFAULT ROUTE----------------------------------#
@app.route('/')
def index():
    return '<br><h2><center>HELLO</center></h2></br>'
#-------------------------CREATION OF A TABLE--------------------------#
@app.route('/ekb-storage_Create', methods=['GET']) 
def ekb_storage_Create():
    try:
        table = request.args.get('table')
        conn = c.connect()
        conn.execute('CREATE TABLE IF NOT EXISTS {} (SNO INTEGER PRIMARY KEY AUTOINCREMENT,DATE TEXT,INJECTION TEXT,WITHDRAWAL TEXT,STORAGE TEXT,URL TEXT);'.format(table))
        message = {"***RESPONSE" : "CREATION OF TABLE IS SUCCESSFUL***"}
        return message
    except Exception as err:
        message = {"***RESPONSE" : "CREATION OF TABLE IS FAILED*** : {}".format(err)}
        return message
    finally:
        conn.close()
#-------------------------INSERTION OF A TABLE--------------------------#
@app.route('/ekb_storage_Insert', methods=['POST'])
def ekb_storage_Insert():
    try:
        table = request.args.get('table')
        conn = c.connect()
        df=scrap()
        df.to_sql(table, conn, if_exists='replace')
        message = {"***RESPONSE" : "INSERTION OF TABLE IS SUCCESSFUL***"}
        return message
    except Exception as err:
        message = {"***RESPONSE" : "INSERTION OF TABLE IS FAILED*** : {}".format(err)}
        return message
    finally:
        conn.close()
#-------------------------SELECTION OF A TABLE--------------------------#
@app.route('/ekb_storage_Select', methods=['GET'])
def ekb_storage_Select():
    try:
        table = request.args.get('table')
        conn = c.connect()
        result = conn.execute("select * from {}".format(table))  
        return json.dumps([dict(r) for r in result])
    except Exception as err:
        message = {"***RESPONSE" : "DATA NOT FOUND*** : {}".format(err)}
        return message
    finally:
        conn.close()
#-------------------------SELECTION OF A PARTICULAR RECORD----------------#
@app.route('/ekb_storage_SelectOne', methods=['GET'])
def ekb_storage_SelectOne():
    try:
        conn = c.connect()
        all_args=request.args.to_dict()
        table=all_args.get('table')
        date=all_args.get('date')
        result = conn.execute("select * from {} where Date = {} ".format(table,date))  
        return json.dumps([dict(r) for r in result])
    except Exception as err:
        message = {"***RESPONSE" : "DATA NOT FOUND*** : {}".format(err)}
        return message
    finally:
        conn.close()
app.run() #RUNNING AN APPLN

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2021 22:46:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 22:46:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Nov/2021 22:46:57] "GET /ekb-storage_Create?table=sai HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/css/bootstrap-tweaks.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/css/prettify.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/js/prettify-min.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/js/default.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/css/default.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/static/js/jquery-1.8.1-min.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Nov/2021 22:46:59] "GET /flask-api/st

In [6]:
#SELENIUM
import os
os.getcwd()
import time
import os
from selenium import webdriver
import pandas as pd
driver_path=r"C:\Users\merit\chromedriver.exe"
driver=webdriver.Chrome(executable_path=driver_path)
driver.maximize_window()
driver.get("https://www.ekb-storage.de/en/ekb-storage/storage-usage/")
driver.find_element_by_xpath("//div[@class='col-sm-6 col-md-7']").click()
driver.find_element_by_xpath("//input[@class='btn btn-primary w-100']").click()
####-------------------------------------------------------------------------####
driver.find_element_by_xpath("//input[@class='btn btn-primary w-100']").click()
#a=driver.find_elements_by_xpath("//div[@class='table-responsive mt-4']//following::tr//following::td")
a=driver.find_elements_by_xpath("//div[@class='table-responsive mt-4']//following::tr")
url="https://www.ekb-storage.de/en/ekb-storage/storage-usage/"
l=[]
for i in a:
    #print(i.text)
    result=i.text.split(" ")
    result.append(url)
    l.append(result)
l=l[1::]
#print(l)
#DATAFRAME
df = pd.DataFrame(l, columns = ["Date","Injection(kWh)","Withdrawal(kWh)","Storage(kWh)","URL"])
df['SNO'] = df.index + 1
df.set_index('SNO',inplace=True)
#--------------------------------------CONVERTING DATA FRAME TO EXCEL-----------------------------------------#
writer = pd.ExcelWriter(r'EKB STORAGE.xlsx', engine='xlsxwriter')
df.to_excel(writer,index=False)
writer.save()
writer.close()


c:\python39\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [7]:
df

,Date,Injection(kWh),Withdrawal(kWh),Storage(kWh),URL
SNO,,,,,
1,2021-10-31,"53,875,024",0,"9,930,060,208",https://www.ekb-storage.de/en/ekb-storage/stor...
2,2021-11-01,"54,518,598","318,110","9,984,260,696",https://www.ekb-storage.de/en/ekb-storage/stor...
3,2021-11-02,"38,040,212","22,200,000","10,000,100,908",https://www.ekb-storage.de/en/ekb-storage/stor...
4,2021-11-03,"9,824,000","39,588,000","9,970,336,908",https://www.ekb-storage.de/en/ekb-storage/stor...
5,2021-11-04,"40,680,000","21,600,000","9,989,416,908",https://www.ekb-storage.de/en/ekb-storage/stor...
6,2021-11-05,"44,586,504","30,800,000","10,003,203,412",https://www.ekb-storage.de/en/ekb-storage/stor...
7,2021-11-06,"67,240,464","21,600,000","10,048,843,876",https://www.ekb-storage.de/en/ekb-storage/stor...
8,2021-11-07,"49,764,694","20,700,000","10,077,908,570",https://www.ekb-storage.de/en/ekb-storage/stor...
9,2021-11-08,"40,686,504","30,660,000","10,089,404,844",https://www.ekb-storage.de/en/ekb-storage/stor...
